In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
from loguru import logger
from matbench.bench import MatbenchBenchmark
from matbench.constants import CLF_KEY

from gptchem.gpt_regressor import GPTRegressor
from gptchem.tuner import Tuner

logger.enable("gptchem")

In [3]:
import time

import decorator


def retry(howmany, *exception_types, **kwargs):
    timeout = kwargs.get("timeout", 0.0)  # seconds

    @decorator.decorator
    def tryIt(func, *fargs, **fkwargs):
        for _ in range(howmany):
            try:
                return func(*fargs, **fkwargs)
            except exception_types or Exception as e:
                print(e)
                if timeout is not None:
                    time.sleep(timeout)

    return tryIt

In [4]:
mb = MatbenchBenchmark(
    autoload=True,
    subset=[
          "matbench_expt_gap",
        #"matbench_steels",
    ],
)

2023-02-01 08:18:44 INFO     Initialized benchmark 'matbench_v0.1' with 1 tasks: 
['matbench_expt_gap']


In [5]:
@retry(3, timeout=5)
def train_test_fold(task, fold):
    regressor = GPTRegressor(
        task.metadata["target"], Tuner(n_epochs=8, learning_rate_multiplier=0.02, wandb_sync=False),
        querier_settings={'max_tokens': 10}
    )
    train_inputs, train_outputs = task.get_train_and_val_data(fold)

    # train and validate your model
    regressor.fit(train_inputs, train_outputs.values)

    # Get testing data
    test_inputs = task.get_test_data(fold, include_target=False)

    # Predict on the testing data
    # Your output should be a pandas series, numpy array, or python iterable
    # where the array elements are floats or bools
    predictions = regressor.predict(test_inputs)

    # Record your data!
    task.record(fold, predictions)
    return predictions

In [6]:
from collections import defaultdict

In [7]:
predictions = defaultdict(list)

for task in mb.tasks:
    task.load()

    for fold_ind, fold in enumerate(task.folds):
        if task.is_recorded[fold_ind]:
            print(f"Skipping fold {fold_ind} of {task.dataset_name}")
            continue
        pred = train_test_fold(task, fold)
        predictions[task.dataset_name].append(pred)
        train_inputs, train_outputs = task.get_train_and_val_data(fold)

    # print(f"{task.dataset_name}: MAE  {task.scores['mae']['mean']}")

2023-02-01 08:18:45 INFO     Dataset matbench_expt_gap already loaded; not reloading dataset.


Upload progress: 100%|██████████| 421k/421k [00:00<00:00, 504Mit/s]


Uploaded file from /Users/kevinmaikjablonka/git/kjappelbaum/gptchem/experiments/04_regression/matbench/out/20230201_081845/train.jsonl: file-ARrYEF7Zdlfsgv10JMANU3jJ


2023-02-01 08:18:48.458 | DEBUG    | gptchem.tuner:tune:188 - Requested fine tuning. {
  "created_at": 1675235928,
  "events": [
    {
      "created_at": 1675235928,
      "level": "info",
      "message": "Created fine-tune: ft-FEYGNp0CwMubNCiqlt3GL8sY",
      "object": "fine-tune-event"
    }
  ],
  "fine_tuned_model": null,
  "hyperparams": {
    "batch_size": null,
    "learning_rate_multiplier": 0.02,
    "n_epochs": 8,
    "prompt_loss_weight": 0.01
  },
  "id": "ft-FEYGNp0CwMubNCiqlt3GL8sY",
  "model": "ada",
  "object": "fine-tune",
  "organization_id": "org-TFRJXw3PPQocOWbu71eI2t9U",
  "result_files": [],
  "status": "pending",
  "training_files": [
    {
      "bytes": 420861,
      "created_at": 1675235927,
      "filename": "/Users/kevinmaikjablonka/git/kjappelbaum/gptchem/experiments/04_regression/matbench/out/20230201_081845/train.jsonl",
      "id": "file-ARrYEF7Zdlfsgv10JMANU3jJ",
      "object": "file",
      "purpose": "fine-tune",
      "status": "uploaded",
      "

2023-02-01 13:32:55 INFO     Recorded fold matbench_expt_gap-0 successfully.


...

2023-02-01 15:14:59.701 | DEBUG    | gptchem.tuner:tune:188 - Requested fine tuning. {
  "created_at": 1675260899,
  "events": [
    {
      "created_at": 1675260899,
      "level": "info",
      "message": "Created fine-tune: ft-dqqViIfMkUYlTss2SFf89pbc",
      "object": "fine-tune-event"
    }
  ],
  "fine_tuned_model": null,
  "hyperparams": {
    "batch_size": null,
    "learning_rate_multiplier": 0.02,
    "n_epochs": 8,
    "prompt_loss_weight": 0.01
  },
  "id": "ft-dqqViIfMkUYlTss2SFf89pbc",
  "model": "ada",
  "object": "fine-tune",
  "organization_id": "org-TFRJXw3PPQocOWbu71eI2t9U",
  "result_files": [],
  "status": "pending",
  "training_files": [
    {
      "bytes": 420263,
      "created_at": 1675260899,
      "filename": "/Users/kevinmaikjablonka/git/kjappelbaum/gptchem/experiments/04_regression/matbench/out/20230201_151457/train.jsonl",
      "id": "file-xaNVbL4q0UW039rbePFw0taS",
      "object": "file",
      "purpose": "fine-tune",
      "status": "uploaded",
      "

2023-02-01 16:54:55 INFO     Recorded fold matbench_expt_gap-2 successfully.


Upload progress: 100%|██████████| 421k/421k [00:00<00:00, 706Mit/s]


Uploaded file from /Users/kevinmaikjablonka/git/kjappelbaum/gptchem/experiments/04_regression/matbench/out/20230201_165455/train.jsonl: file-95TFqfcTknod0DTizJ4ZbXXQ


2023-02-01 16:54:58.029 | DEBUG    | gptchem.tuner:tune:188 - Requested fine tuning. {
  "created_at": 1675266897,
  "events": [
    {
      "created_at": 1675266897,
      "level": "info",
      "message": "Created fine-tune: ft-TUSbWrEwLd6dZP9CYYS9FXso",
      "object": "fine-tune-event"
    }
  ],
  "fine_tuned_model": null,
  "hyperparams": {
    "batch_size": null,
    "learning_rate_multiplier": 0.02,
    "n_epochs": 8,
    "prompt_loss_weight": 0.01
  },
  "id": "ft-TUSbWrEwLd6dZP9CYYS9FXso",
  "model": "ada",
  "object": "fine-tune",
  "organization_id": "org-TFRJXw3PPQocOWbu71eI2t9U",
  "result_files": [],
  "status": "pending",
  "training_files": [
    {
      "bytes": 420723,
      "created_at": 1675266897,
      "filename": "/Users/kevinmaikjablonka/git/kjappelbaum/gptchem/experiments/04_regression/matbench/out/20230201_165455/train.jsonl",
      "id": "file-95TFqfcTknod0DTizJ4ZbXXQ",
      "object": "file",
      "purpose": "fine-tune",
      "status": "uploaded",
      "

In [19]:
task.scores

ValueError: invalid __array_struct__

In [ ]:
mb.to_file("gpt_expt_gap_bench.json.gz")

2023-01-30 18:53:04 INFO     Successfully wrote MatbenchBenchmark to file 'gpt_steel_bench.json.gz'.
